In [86]:
import numpy as np
import src.util as util

from src.linear_model import LinearModel

## (a) : Show Poission dist is exponential family

$$\begin{align}
p(y;\lambda) &= \frac{e^{-\lambda} \lambda ^{y}}{y!} \\
&= \exp(\log(\frac{\lambda ^{y} e^{-\lambda} }{y!})) \\
&= \exp(y\log(\lambda ) -\lambda  - \log(y!)) \\
&= \frac{1}{y!} \exp(y\log(\lambda ) -\lambda ) \\
\\
&\text{Then we put into cannoical form} : b(y) \exp(\eta T(y) -a(\eta)) \\
\\ 
&=  \frac{1}{y!} \exp(\eta y -a(\lambda) ) \\ \\
&\text{Where} \\ \\
&b(y) = \frac{1}{y!} , \eta = \log(\lambda) , T(y)=y, a(\eta)= e^{\eta} = \lambda

\end{align}$$


### Note:

__*__ why $\exp(-\log(y)) = 1/y$:

1. Let's start with the definition of the natural logarithm: $\log(y) = x$ if and only if $e^x = y$.
2. From this definition, we can see that $e^{\log(y)} = y$.
3. Now, let's take the reciprocal of both sides: $\frac{1}{e^{\log(y)}} = \frac{1}{y}$.
4. Using the property that $a^{-b} = \frac{1}{a^b}$, we can rewrite the left side as $e^{-\log(y)} = \frac{1}{y}$.


## (b) : find the canonical response function for poission family

Canonical response function's inverse, $g^{−1}$, is called the **canonical link function**, thus we need to find canonical link function then reverse it to get the answer.

The **canonical link function** is the default link function used in a generalized linear model (GLM) for a particular distribution. It relates the expected value of the response variable to the linear predictor. For distributions in the exponential family, the canonical link function is the function that relates the mean of the distribution to the natural parameter of the distribution.

The **inverse link function**, on the other hand, is simply the inverse of the link function. It maps the linear predictor back to the expected value of the response variable. The inverse of the canonical link function is called the **canonical response function**.

Poisson regression, where the response variable follows a Poisson distribution, the canonical link function is the logarithm and the canonical response function (i.e., the inverse link function) is the exponential function. This means that if we have a linear predictor $\eta = \theta^TX$, where $X$ is a matrix of predictor variables and $\theta$ is a vector of coefficients, then we can write $\log(\mu) = \eta$, where $\mu$ is the expected value of the response variable. The inverse link function maps $\eta$ back to $\mu$, so we have $\mu = \exp(\eta)$.



## (c) derive stochastic gradient ascent update rule

We can write the probability function as :

$$
p(y^{(i)}| x^{(i)}, \theta) = \prod^{n}_{i=1} \frac{\exp(y^{(i)} \theta^T x^{(i)} - e^{\theta^T x^{(i)}})}{y^{(i)}!}
$$

then take the log likelyhood we convert product into sum:

$$
\mathcal{L}(y^{(i)} | x^{(i)}, \theta) = \sum^{n}_{i=1} ( y^{(i)} \theta^T x^{(i)}  - \exp( \theta^T x^{(i)}) - \log(y^{(i)}!) )
$$

to get maximum likelyhood with respect to $\theta$, we derivetive the likelyhood function w.r.t $\theta$ and set to 0:

$$\begin{align}
\frac{\partial \mathcal{L} (y^{(i)} | x^{(i)}, \theta)}{\partial \theta} &= \sum^{n}_{i=1} (y^{(i)} x^{(i)} - \exp( \theta^T x^{(i)}) x^{(i)}) \\
&= \sum^{n}_{i=1} (y^{(i)} - e^{( \theta^T x^{(i)})}) x^{(i)} = 0
\end{align}$$

However, this equation cannot be solved analytically for $\theta$. Instead, numerical methods such as Newton-Raphson or gradient descent are typically used to find the maximum likelihood estimate of $\theta$. Thus, this equation can be write as:

$$
\frac{\partial \mathcal{L} (y^{(i)} | x^{(i)}, \theta)}{\partial \theta} = \sum^{n}_{i=1} (y^{(i)} - \hat{y}^{(i)}) x^{(i)} = gradient
$$


## (d) implement Poission regression

In [87]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))


def prob_d(lr, train_path, eval_path, pred_path):
    """Problem 3(d): Poisson regression with gradient ascent.

    Args:
        lr: Learning rate for gradient ascent.
        train_path: Path to CSV file containing dataset for training.
        eval_path: Path to CSV file containing dataset for evaluation.
        pred_path: Path to save predictions.
    """
    # Load training set
    x_train, y_train = util.load_dataset(train_path, add_intercept=False)
    x_eval, y_eval = util.load_dataset(eval_path, add_intercept=False)
    # *** START CODE HERE ***
    model = PoissonRegression(step_size=lr, max_iter=10**3)
    model.fit(x_train, y_train)
    preds = model.predict(x_eval)
    print(rmse(y_eval, preds))
    # *** END CODE HERE ***


class PoissonRegression(LinearModel):
    """Poisson Regression.

    Example usage:
        > clf = PoissonRegression(step_size=lr)
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def fit(self, x, y):
        """Run gradient ascent to maximize likelihood for Poisson regression.

        Args:
            x: Training example inputs. Shape (m, n).
            y: Training example labels. Shape (m,).
        """
        # *** START CODE HERE ***
    
        m, n = x.shape
        if self.theta == None:
            self.theta =np.random.rand(n)
            # self.theta = np.zeros(n) 

        for _ in range(self.max_iter):
            y_hat = np.exp(x@self.theta)
            grad = x.T @ (y - y_hat)
            lr = self.step_size * grad
            self.theta += lr
        # *** END CODE HERE ***

    def predict(self, x):
        """Make a prediction given inputs x.

        Args:
            x: Inputs of shape (m, n).

        Returns:
            Floating-point prediction for each input, shape (m,).
        """
        # *** START CODE HERE ***
        y_hat = np.exp(x @ self.theta)
        return y_hat
        # *** END CODE HERE ***


prob_d(8.5e-11, "./data/ds4_train.csv","./data/ds4_valid.csv","")

1942.9024896371197
